In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNet, DenseNet121, VGG19
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define constants
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
NUM_CLASSES = 2  # Adjust according to your dataset, e.g., "normal" and "abnormal"

# Load and preprocess data
datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    validation_split=0.2,
    horizontal_flip=True,
    zoom_range=0.2
)

train_generator = datagen.flow_from_directory(
    'path/to/dataset',
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

val_generator = datagen.flow_from_directory(
    'path/to/dataset',
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

# Function to build and compile models
def build_model(base_model):
    # Add global average pooling layer
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    # Add fully connected layer
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)
    # Add final output layer
    predictions = Dense(NUM_CLASSES, activation='softmax')(x)

    # Compile model
    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Create MobileNet model
base_model_mobilenet = MobileNet(weights='imagenet', include_top=False, input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3))
mobilenet_model = build_model(base_model_mobilenet)

# Create DenseNet121 model
base_model_densenet = DenseNet121(weights='imagenet', include_top=False, input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3))
densenet_model = build_model(base_model_densenet)

# Create VGG19 model
base_model_vgg19 = VGG19(weights='imagenet', include_top=False, input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3))
vgg19_model = build_model(base_model_vgg19)

# Train models
EPOCHS = 10

# Training MobileNet model
mobilenet_model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=val_generator
)

# Training DenseNet121 model
densenet_model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=val_generator
)

# Training VGG19 model
vgg19_model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=val_generator
)
